# DuckDB Extraction Pipeline (0-48h Feature Build)

This notebook extracts early (first 48h) multimodal data for the cohort in `data/initial_cohort.csv` using a local DuckDB database (`data/mimicii.duckdb`), applies the same inclusion criteria (first admission, LOS ≥54h), aggregates features via `project/features.py`, and saves standardized artifacts into `project/artifacts/`:
- `features_full.parquet`
- `feature_columns.json`
- `feature_provenance.json`

**Adjustable parameters:** set a subject sample limit or run full cohort.

In [1]:
# Environment & Imports (local)
import os, sys, pathlib
from importlib.util import find_spec

# Add project root to path if notebook is under notebooks/
ROOT = pathlib.Path(__file__).resolve().parents[1] if '__file__' in globals() else pathlib.Path.cwd().parents[0]
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))

missing = []
for pkg in ['duckdb','pandas','matplotlib']:
    if find_spec(pkg) is None: missing.append(pkg)
if missing:
    print('Missing packages detected:', missing)
    print('Install them (example): pip install ' + ' '.join(missing))

import pandas as pd
import matplotlib.pyplot as plt
try:
    import duckdb
except ModuleNotFoundError:
    raise RuntimeError('duckdb not installed. Please install and re-run.')

from project import run_pipeline_on_unseen_data
print('Imports ready.')

Imports ready.


In [2]:
# Imports & configuration
from pathlib import Path
import pandas as pd, numpy as np
import duckdb, json, math, sys, datetime
from importlib import reload
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'project' / 'artifacts'
DB_PATH = DATA_DIR / 'mimiciii.duckdb'  # Adjust if your filename differs
SUBJECT_SAMPLE_LIMIT = None  # e.g., 500 for a quick run; set None for full
RANDOM_SEED = 42
print('PROJECT_ROOT:', PROJECT_ROOT)
print('DuckDB path exists:', DB_PATH.exists())
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_ROOT: c:\Users\Almog Luz\Documents\GitHub\mlhc-final-project
DuckDB path exists: True


## 1. Load Cohort

In [3]:
cohort_path = DATA_DIR / 'initial_cohort.csv'
if not cohort_path.exists():
    raise FileNotFoundError(f'Missing cohort file: {cohort_path}')
cohort = pd.read_csv(cohort_path)
if 'subject_id' not in cohort.columns:
    raise ValueError('`subject_id` column missing in cohort CSV')
cohort = cohort.dropna(subset=['subject_id'])
cohort['subject_id'] = cohort['subject_id'].astype(int)
if SUBJECT_SAMPLE_LIMIT is not None:
    cohort = cohort.sample(n=min(SUBJECT_SAMPLE_LIMIT, len(cohort)), random_state=RANDOM_SEED)
print('Cohort size (after optional sample):', len(cohort))
cohort.head()

Cohort size (after optional sample): 32513


,subject_id
0,22392
1,2847
2,12056
3,25600
4,73125


## 2. Connect to DuckDB

In [4]:
con = duckdb.connect(str(DB_PATH))
print('Connected to DuckDB.')
# (Optional) list tables to verify schema
try:
    tbls = con.execute("SHOW TABLES").fetchdf()
    print('Available tables:', tbls['name'].tolist())
except Exception as e:
    print('Could not list tables:', e)

Connected to DuckDB.
Available tables: ['ADMISSIONS', 'CALLOUT', 'CAREGIVERS', 'CHARTEVENTS', 'CPTEVENTS', 'DATETIMEEVENTS', 'DIAGNOSES_ICD', 'DRGCODES', 'D_CPT', 'D_ICD_DIAGNOSES', 'D_ICD_PROCEDURES', 'D_ITEMS', 'D_LABITEMS', 'ICUSTAYS', 'INPUTEVENTS_CV', 'INPUTEVENTS_MV', 'LABEVENTS', 'MICROBIOLOGYEVENTS', 'NOTEEVENTS', 'OUTPUTEVENTS', 'PATIENTS', 'PRESCRIPTIONS', 'PROCEDUREEVENTS_MV', 'PROCEDURES_ICD', 'SERVICES', 'TRANSFERS']


## 3. Extract Admissions (First Admission + LOS Filter)

In [5]:
sid_csv = ','.join(map(str, cohort['subject_id'].tolist()))
adm_sql = f"""
SELECT subject_id, hadm_id, admittime, dischtime, deathtime, admission_type,
       admission_location, discharge_location, diagnosis, insurance, language,
       marital_status, ethnicity
FROM admissions
WHERE subject_id IN ({sid_csv})
ORDER BY subject_id, admittime
"""
admissions_all = con.execute(adm_sql).fetchdf()
print('Raw admissions rows:', len(admissions_all))
if admissions_all.empty:
    raise RuntimeError('Admissions query returned zero rows; verify DuckDB schema and table population.')
# Normalize column names to lower for safety
admissions_all.columns = [c.lower() for c in admissions_all.columns]
first_adm = (admissions_all.sort_values(['subject_id','admittime'])
    .groupby('subject_id', as_index=False)
    .first())
first_adm['admittime'] = pd.to_datetime(first_adm['admittime'])
first_adm['dischtime'] = pd.to_datetime(first_adm['dischtime'])
first_adm['los_hours'] = (first_adm['dischtime'] - first_adm['admittime']).dt.total_seconds()/3600.0
pre_filter_n = len(first_adm)
first_adm = first_adm[first_adm['los_hours'] >= 54].copy()
print(f'First admissions retained >=54h: {len(first_adm)} / {pre_filter_n}')
hadm_ids = first_adm['hadm_id'].dropna().astype(int).tolist()
hadm_csv = ','.join(map(str, hadm_ids)) if hadm_ids else '-1'
first_adm.head()

Raw admissions rows: 41244
First admissions retained >=54h: 28473 / 32513


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,diagnosis,insurance,language,marital_status,ethnicity,los_hours
0,1000,143040,2144-01-19 20:15:00,2144-02-25 06:05:00,2144-02-25 06:05:00,URGENT,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,HEART FAILURE,Private,None,MARRIED,UNKNOWN/NOT SPECIFIED,873.833333
1,10000,187813,2186-08-10 21:57:00,2186-08-31 17:00:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,END STAGE LIVER DISEASE;END STAGE KIDNEY DISEASE,Self Pay,SPAN,None,HISPANIC OR LATINO,499.050000
3,10003,144039,2111-11-21 23:47:00,2111-12-02 16:34:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,HEMOTHORAX/RIB FX,Private,None,MARRIED,BLACK/AFRICAN AMERICAN,256.783333
4,10004,164713,2182-11-02 13:27:00,2182-11-19 16:30:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,S/P FALL;ICH,Medicare,ENGL,MARRIED,WHITE,411.050000
5,10005,181146,2155-02-27 06:00:00,2155-03-01 13:40:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,NEWBORN,Medicaid,None,SINGLE,WHITE,55.666667


## 3a. Label Regeneration & Cohort Filtering (Enforce 48h + 6h Gap)
This step rebuilds labels early (DuckDB) so that all subsequent modality extraction & feature engineering only operate on temporally valid subjects:
- Keeps only first admissions with LOS >= 54h
- Excludes subjects with death, discharge, or next admission (readmission) within 54h of index admission
Outputs:
- `project/artifacts/labels.csv`
- In-memory filtered `first_adm` and `subject_ids` used downstream.

If you adjust inclusion criteria, re-run this cell to refresh downstream derivations.

In [ ]:
# Build labels early & filter cohort
from project.labels import build_labels_duckdb
import pandas as pd, pathlib, duckdb

DB_PATH = pathlib.Path('../data/mimiciii.duckdb')
con = duckdb.connect(str(DB_PATH), read_only=True)

# Use current cohort subject_ids
subject_ids = first_adm['subject_id'].astype(int).tolist()
labels_df = build_labels_duckdb(con, subject_ids)
print(f"Labels built: {len(labels_df)} subjects (after LOS>=54h & early-event exclusion)")

# Persist labels
labels_path = pathlib.Path('../project/artifacts/labels.csv')
labels_path.parent.mkdir(parents=True, exist_ok=True)
labels_df.to_csv(labels_path, index=False)
print('Wrote labels to', labels_path)

# Filter first_adm and update subject_ids for downstream extractions
keep = set(labels_df['subject_id'].tolist())
pre = len(first_adm)
first_adm = first_adm[first_adm['subject_id'].isin(keep)].copy()
print(f'Filtered first_adm: {pre} -> {len(first_adm)}')
subject_ids = first_adm['subject_id'].astype(int).tolist()
print('Subject IDs prepared for downstream modality extraction:', len(subject_ids))

## 4. Extract Early (0–48h) Modalities

In [6]:
# Use original BigQuery extraction functions directly (no local rewrites)
# NOTE: These require a configured BigQuery client (`google.cloud.bigquery.Client`).
# If you only have DuckDB locally, skip executing this cell or adapt a separate local path.
from project.extract import (
    get_first_admissions,
    get_demographics,
    get_vitals_48h,
    get_labs_48h,
    get_prescriptions_48h,
    get_procedures_48h,
)

try:
    from google.cloud import bigquery as bq  # type: ignore
except ImportError:
    raise RuntimeError("google-cloud-bigquery not installed. Install it to run original extract.py functions.")

# Instantiate client (assumes credentials are configured in environment, e.g. GOOGLE_APPLICATION_CREDENTIALS)
client = bq.Client()

subject_ids = cohort['subject_id'].astype(int).tolist()
first_adm = get_first_admissions(client, subject_ids)
if first_adm.empty:
    raise RuntimeError('No admissions returned from BigQuery for provided subject_ids.')

# Compute LOS and apply >=54h filter (mirroring earlier logic)
first_adm['admittime'] = pd.to_datetime(first_adm['admittime'])
first_adm['dischtime'] = pd.to_datetime(first_adm['dischtime'])
first_adm['los_hours'] = (first_adm['dischtime'] - first_adm['admittime']).dt.total_seconds()/3600.0
pre_filter = len(first_adm)
first_adm = first_adm[first_adm['los_hours'] >= 54].copy()
print(f'First admissions retained (LOS>=54h): {len(first_adm)} / {pre_filter}')

hadm_ids = first_adm['hadm_id'].dropna().astype(int).tolist()

# Modalities
print('Querying demographics...')
demo = get_demographics(client, subject_ids)
print('Querying vitals...')
vitals = get_vitals_48h(client, hadm_ids)
print('Querying labs...')
labs = get_labs_48h(client, hadm_ids)
print('Querying prescriptions...')
rx = get_prescriptions_48h(client, hadm_ids)
print('Querying procedures...')
proc = get_procedures_48h(client, hadm_ids)

mods = {'demo': demo, 'vitals': vitals, 'labs': labs, 'rx': rx, 'proc': proc}
print('Modalities non-empty flags:', {k: (not v.empty) for k,v in mods.items()})

# Optional sanity check: window adherence in vitals
if not first_adm.empty and not vitals.empty and 'charttime' in vitals.columns:
    sample = vitals.head(10000).merge(first_adm[['hadm_id','admittime']], on='hadm_id', how='left')
    sample['charttime'] = pd.to_datetime(sample['charttime'], errors='coerce')
    delta_hr = (sample['charttime'] - sample['admittime']).dt.total_seconds()/3600.0
    if (delta_hr.dropna() > 48).any():
        print('WARNING: Observed vital rows outside 0-48h range.')

print('Extraction via original extract.py functions complete.')

First admissions retained (LOS>=54h): 28473 / 32513
Querying demographics...
Querying vitals...
Querying vitals...
Querying labs...
Querying labs...
Querying prescriptions...
Querying prescriptions...
Querying procedures...
Querying procedures...
Modalities non-empty flags: {'demo': True, 'vitals': True, 'labs': True, 'rx': True, 'proc': True}
Extraction via original extract.py functions complete.
Modalities non-empty flags: {'demo': True, 'vitals': True, 'labs': True, 'rx': True, 'proc': True}
Extraction via original extract.py functions complete.


## 5. Build Feature Matrix

In [7]:
# Build feature matrix from extracted modalities
from project.features import build_features

# If vitals missing hadm_id but have subject_id, proceed (aggregation uses subject_id)
if not vitals.empty and 'hadm_id' not in vitals.columns:
    print('Vitals missing hadm_id; continuing with subject_id-based aggregation.')

if first_adm.empty:
    print('No admissions after LOS filter; skipping feature build.')
    features = pd.DataFrame()
else:
    features = build_features(
        first_adm,
        demo,
        vitals,
        labs,
        rx,
        proc,
    )
    print('Feature matrix shape:', features.shape)
    leakage_cols = [c for c in features.columns if 'los_hours' in c.lower()]
    if leakage_cols:
        print('Dropping leakage columns:', leakage_cols)
        features = features.drop(columns=leakage_cols)
features.head(3)

DEBUG(build_features): first_adm columns -> ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'diagnosis', 'insurance', 'language', 'marital_status', 'ethnicity', 'los_hours']
DEBUG(build_features): demo columns -> ['subject_id', 'gender', 'dob', 'dod', 'expire_flag']
Feature matrix shape: (28473, 1429)
Feature matrix shape: (28473, 1429)


,age,gender_M,gender_F,eth_ASIAN,eth_BLACK,eth_HISPANIC,eth_OTHER,eth_WHITE,aprotinin cc/hr__mean,aprotinin ml/hr__mean,...,0,1,2,3,proc_vent_any,proc_rrt_any,proc_central_line_any,admit_hour,admit_weekend,admit_month
subject_id,,,,,,,,,,,,,,,,,,,,,
2,0.002175,1,0,1,0,0,0,0,NaN,NaN,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,19,0,7
3,76.525112,1,0,0,0,0,0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,0,10
4,47.843996,0,1,0,0,0,0,1,NaN,NaN,...,1.0,1.0,0.0,1.0,NaN,NaN,NaN,0,0,3


## 6. Persist Artifacts

In [8]:
feat_path = ARTIFACTS_DIR / 'features_full.parquet'
prov_path = ARTIFACTS_DIR / 'feature_provenance.json'
cols_path = ARTIFACTS_DIR / 'feature_columns.json'
features.to_parquet(feat_path)
from project.features import build_feature_provenance
prov = build_feature_provenance(features)
prov_path.write_text(json.dumps(prov, indent=2))
cols_path.write_text(json.dumps(list(features.columns), indent=2))
print('Saved:')
for p in [feat_path, prov_path, cols_path]:
    print('  -', p.resolve())

Saved:
  - C:\Users\Almog Luz\Documents\GitHub\mlhc-final-project\project\artifacts\features_full.parquet
  - C:\Users\Almog Luz\Documents\GitHub\mlhc-final-project\project\artifacts\feature_provenance.json
  - C:\Users\Almog Luz\Documents\GitHub\mlhc-final-project\project\artifacts\feature_columns.json


## 7. Basic Feature Diagnostics

In [9]:
# Missingness overview (top 15 most-missing features)
if features.empty:
    print('Feature matrix empty – nothing to diagnose.')
else:
    miss = features.isna().mean().sort_values(ascending=False)[:15]
    display(miss.to_frame('missing_fraction'))
    # Simple distribution snapshot for a few continuous features
    sample_cols = [c for c in features.columns if features[c].dtype != 'O'][:8]
    if sample_cols:
        desc = features[sample_cols].describe().T
        display(desc)
print('Done.')

,missing_fraction
ca gluc gm/hr__range,0.999965
cisatracu mg/kg/hr__mean,0.999965
mucomyst mg/hr__range,0.999965
mucomyst mg/hr__mean,0.999965
ca gluc gm/hr__mean,0.999965
solumedrol mg/hr__count,0.999965
mucomyst mg/hr__last,0.999965
nicardipine mg/hr__last,0.999965
nicardipine--mg/hr__last,0.999965
"hematocrit, other fluid__std",0.999965


,count,mean,std,min,25%,50%,75%,max
age,28473.0,53.335578,27.003550,0.0,42.083037,61.623251,73.49479,90.0
gender_M,28473.0,0.565694,0.495674,0.0,0.000000,1.000000,1.00000,1.0
gender_F,28473.0,0.434306,0.495674,0.0,0.000000,0.000000,1.00000,1.0
eth_ASIAN,28473.0,0.032346,0.176922,0.0,0.000000,0.000000,0.00000,1.0
eth_BLACK,28473.0,0.080041,0.271361,0.0,0.000000,0.000000,0.00000,1.0
eth_HISPANIC,28473.0,0.034278,0.181946,0.0,0.000000,0.000000,0.00000,1.0
eth_OTHER,28473.0,0.151723,0.358758,0.0,0.000000,0.000000,0.00000,1.0
eth_WHITE,28473.0,0.701612,0.457559,0.0,0.000000,1.000000,1.00000,1.0


Done.
